#### style

In [8]:
%%html
<style>
.container {width: 90% !important; }
#logo
{
    float: left;
}
#names
{
    float: right;
    
}
#logo2
{
    float: right;
    height=100;
    width=100;
    margin: 5px;
}
#top_menu
{
    margin: 20px auto 0;
    
    height:300px;
    width: 400px;
}
</style>

# Part 2 - Pandas Demo

## Data Wrangling

- We need to make the data fit our needs
- Data wrangling is the process of doing this

### Goal:

1. Each variable forms a column
2. Each observation forms a row
3. Each type of observational unit forms a table

In [1]:
import os
import pandas as pd

### Read Data From An HTML Table

In [3]:
# %load solutions/part2_demo/sol1.py
fp = 'data/nba.csv'

if not os.path.exists(fp):
    tables = pd.read_html("http://www.basketball-reference.com/leagues/NBA_2016_games.html")
    games = tables[0]
    games.to_csv(fp)
else:
    games = pd.read_csv(fp)
games.head()


Unnamed: 0               Date Start (ET)       Visitor/Neutral  PTS  \
0           0  Tue, Oct 27, 2015      8:00p       Detroit Pistons  106   
1           1  Tue, Oct 27, 2015      8:00p   Cleveland Cavaliers   95   
2           2  Tue, Oct 27, 2015     10:30p  New Orleans Pelicans   95   
3           3  Wed, Oct 28, 2015      7:30p    Philadelphia 76ers   95   
4           4  Wed, Oct 28, 2015      7:30p         Chicago Bulls  115   

            Home/Neutral  PTS.1 Unnamed: 6 Unnamed: 7  Attend.  Notes  
0          Atlanta Hawks     94  Box Score        NaN    19187    NaN  
1          Chicago Bulls     97  Box Score        NaN    21957    NaN  
2  Golden State Warriors    111  Box Score        NaN    19596    NaN  
3         Boston Celtics    112  Box Score        NaN    18624    NaN  
4          Brooklyn Nets    100  Box Score        NaN    17732    NaN

### Clean Up the Column Names

- Leverage method chaining
- Removed unwanted information
- Normalize column names
- `dropna` has a thresh argument
    + If at least (4) the thresh items are missing, the row is dropped. 
    + It was used it to remove the "Month headers" that slipped into the table.
- `assign` can take a callable
    + This lets us refer to the DataFrame in the previous step of the chain. 
- `set_index` has an append keyword. 
    + keep the original index around since it will be our unique identifier per game.
- Use `rename_axis` to set the index names

In [5]:
# %load solutions/part2_demo/sol2.py
column_names = {'Date': 'date', 'Start (ET)': 'start',
                'Unamed: 2': 'box', 'Visitor/Neutral': 'away_team', 
                'PTS': 'away_points', 'Home/Neutral': 'home_team',
                'PTS.1': 'home_points', 'Unamed: 7': 'n_ot'}

games = (
             games
             .rename(columns=column_names)
             .rename(columns=str.lower)
             .dropna(thresh=4)
             [['date', 'away_team', 'away_points', 'home_team', 'home_points']]
             .assign(date=lambda x: pd.to_datetime(x['date'], format='%a, %b %d, %Y'))
             .set_index('date', append=True) 
             .rename_axis(["game_id", "date"])
             .sort_index()
        )
games.head()


away_team  away_points              home_team  \
game_id date                                                                   
0       2015-10-27       Detroit Pistons          106          Atlanta Hawks   
1       2015-10-27   Cleveland Cavaliers           95          Chicago Bulls   
2       2015-10-27  New Orleans Pelicans           95  Golden State Warriors   
3       2015-10-28    Philadelphia 76ers           95         Boston Celtics   
4       2015-10-28         Chicago Bulls          115          Brooklyn Nets   

                    home_points  
game_id date                     
0       2015-10-27           94  
1       2015-10-27           97  
2       2015-10-27          111  
3       2015-10-28          112  
4       2015-10-28          100

### Answer a Question!

<h1>How many days of rest did each team get between each game?</h1>

#### Ask: Is my dataset **TIDY**?

- Since our observation would be a `(team, game)` pair, we don't have this yet


In [8]:
# %load solutions/part2_demo/sol3.py
tidy = pd.melt(games.reset_index(),
               id_vars=['game_id', 'date'], 
               value_vars=['away_team', 'home_team'],
               value_name='team')
tidy.head()

game_id       date   variable                  team
0        0 2015-10-27  away_team       Detroit Pistons
1        1 2015-10-27  away_team   Cleveland Cavaliers
2        2 2015-10-27  away_team  New Orleans Pelicans
3        3 2015-10-28  away_team    Philadelphia 76ers
4        4 2015-10-28  away_team         Chicago Bulls

#### Are we `TIDY` Now?

- It meets the rules defined before us!
    + Each variable is in a column
    + Each observation is on its own row
    + Each type of observation unit for a table

#### Rethink the Question

`"How many days of rest between games"` to operation `"date of today's game - date of previous game - 1"`

In [10]:
# %load solutions/part2_demo/sol4.py
tidy['rest'] = (
                tidy
                .sort_values('date')
                .groupby('team')
                .date.diff().dt.days - 1
               )
tidy.dropna().head()


game_id       date   variable                  team  rest
4         4 2015-10-28  away_team         Chicago Bulls   0.0
8         8 2015-10-28  away_team   Cleveland Cavaliers   0.0
14       14 2015-10-28  away_team  New Orleans Pelicans   0.0
17       17 2015-10-29  away_team     Memphis Grizzlies   0.0
18       18 2015-10-29  away_team      Dallas Mavericks   0.0

## Visualize the Results

In [6]:
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='ticks', context='paper')
warnings.filterwarnings("ignore")

In [7]:
g = sns.FacetGrid(tidy, col='team', col_wrap=6, hue='team', height=2)
g.map(sns.barplot, 'variable', 'rest')

## Onto Part 3!